# Project Setup

In [27]:
import os
import numpy as np
import pandas as pd
import pickle
import quandl
from datetime import datetime

import seaborn as sns
import time
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)


from talib.abstract import *
import talib


from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import mutual_info_regression

from sklearn import preprocessing
from sklearn import decomposition

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_score   
    
from sklearn.ensemble import VotingClassifier   
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier    
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB


quandl.ApiConfig.api_key = 'RtzgMLo48kkxxh9-sLuf'
data_dir = './data/'

## Pull BTC Exchange Pricing Data
The nature of Bitcoin exchanges is that the pricing is determined by supply and demand, hence no single exchange contains a true "master price" of Bitcoin.  To solve this issue, along with that of down-spikes, we'll pull data from three more major Bitcoin changes to calculate an aggregate Bitcoin price index.

In [28]:
def get_quandl_data(quandl_id):
    '''Download and cache Quandl dataseries'''
    cache_path = '{}{}.pkl'.format(data_dir, quandl_id.replace('/','-'))
    try:
        f = open(cache_path, 'rb')
        df = pickle.load(f)   
        print('Loaded {} from cache'.format(quandl_id))
    except (OSError, IOError) as e:
        print('Downloading {} from Quandl'.format(quandl_id))
        df = quandl.get(quandl_id, returns="pandas", start_date="2010-01-01")
        df.to_pickle(cache_path)
        print('Cached {} at {}'.format(quandl_id, cache_path))
    return df

def merge_dfs_on_column(dataframes, labels, col):
    '''Merge a single column of each dataframe into a new combined dataframe'''
    series_dict = {}
    for index in range(len(dataframes)):
        series_dict[labels[index]] = dataframes[index][col]
        
    return pd.DataFrame(series_dict)

In [29]:
# Pull pricing data for 3 more BTC exchanges
exchanges = ['COINBASE','BITSTAMP','ITBIT', 'KRAKEN']

exchange_data = {}

for exchange in exchanges:
    exchange_code = 'BCHARTS/{}USD'.format(exchange)
    btc_exchange_df = get_quandl_data(exchange_code)
    exchange_data[exchange] = btc_exchange_df

btc_usd_df = pd.DataFrame()

# Merge the BTC price dataseries' into a single dataframe
btc_usd_df['weightedAverage'] = merge_dfs_on_column(list(exchange_data.values()),
                                       list(exchange_data.keys()),
                                       'Weighted Price').replace(0,np.nan).mean(axis=1)  
btc_usd_df['quoteVolume'] = merge_dfs_on_column(list(exchange_data.values()),
                                       list(exchange_data.keys()),
                                       'Volume (Currency)') .replace(0, np.nan).mean(axis=1)
btc_usd_df['volume'] = merge_dfs_on_column(list(exchange_data.values()),
                                       list(exchange_data.keys()),
                                       'Volume (BTC)').replace(0, np.nan).mean(axis=1)
btc_usd_df['close'] = merge_dfs_on_column(list(exchange_data.values()),
                                       list(exchange_data.keys()),
                                       'Close').replace(0, np.nan).mean(axis=1)
btc_usd_df['low'] = merge_dfs_on_column(list(exchange_data.values()),
                                       list(exchange_data.keys()),
                                       'Low').replace(0, np.nan).mean(axis=1)
btc_usd_df['high'] = merge_dfs_on_column(list(exchange_data.values()),
                                       list(exchange_data.keys()),
                                       'High').replace(0, np.nan).mean(axis=1)
btc_usd_df['open'] = merge_dfs_on_column(list(exchange_data.values()),
                                       list(exchange_data.keys()),
                                       'Open').replace(0, np.nan).mean(axis=1)

# Plot the average BTC price
btc_trace = go.Scatter(x=btc_usd_df.index, y=btc_usd_df['weightedAverage'])
py.iplot([btc_trace])


btc_usd_df.dropna(inplace=True)

btc_usd_df.head()

Loaded BCHARTS/COINBASEUSD from cache
Loaded BCHARTS/BITSTAMPUSD from cache
Loaded BCHARTS/ITBITUSD from cache
Loaded BCHARTS/KRAKENUSD from cache


,weightedAverage,quoteVolume,volume,close,low,high,open
Date,,,,,,,
2011-09-13,5.929231,346.097389,58.371382,5.97,5.65,6.00,5.80
2011-09-14,5.590798,341.854813,61.145984,5.53,5.52,5.72,5.58
2011-09-15,5.094272,408.259002,80.140795,5.13,5.00,5.24,5.12
2011-09-16,4.854515,193.763147,39.914007,4.85,4.80,4.87,4.82
2011-09-17,4.870000,1.461000,0.300000,4.87,4.87,4.87,4.87


# Stock market OHLC data

In [30]:
visa_stock = get_quandl_data('EOD/V')[['Adj_Open', 'Adj_High', 'Adj_Low', 'Adj_Close', 'Adj_Volume']]

visa_stock = visa_stock.rename(columns={'Adj_Open': 'open'})
visa_stock = visa_stock.rename(columns={'Adj_High': 'high'})
visa_stock = visa_stock.rename(columns={'Adj_Low': 'low'})
visa_stock = visa_stock.rename(columns={'Adj_Close': 'close'})
visa_stock = visa_stock.rename(columns={'Adj_Volume': 'volume'})

visa_stock['weightedAverage'] = (visa_stock['low'] + visa_stock['high'])/2
visa_stock['quoteVolume'] = visa_stock['volume']

visa_stock.head()

Loaded EOD/V from cache


,open,high,low,close,volume,weightedAverage,quoteVolume
Date,,,,,,,
2010-01-04,20.733227,20.914643,20.606000,20.766212,20180000.0,20.760321,20180000.0
2010-01-05,20.568303,20.622493,20.262017,20.528251,25833600.0,20.442255,25833600.0
2010-01-06,20.499978,20.516470,20.238457,20.252593,16254000.0,20.377464,16254000.0
2010-01-07,20.254949,20.497622,20.155995,20.441077,27841200.0,20.326808,27841200.0
2010-01-08,20.455213,20.589508,20.316206,20.497622,11907200.0,20.452857,11907200.0


# Retrieve Altcoin Pricing Data
Now that we have a solid time series dataset for the price of Bitcoin, let's pull in some data on non-Bitcoin cryptocurrencies, commonly referred to as altcoins.

## Define Poloniex API Helper Functions

For retrieving data on cryptocurrencies we'll be using the [Poloniex API](https://poloniex.com/support/api/).  To assist in the altcoin data retrieval, we'll define two helper functions to download and cache JSON data from this API.

First, we'll define `get_json_data`, which will download and cache JSON data from a provided URL.

In [31]:
def get_json_data(json_url, file_name):
    '''Download and cache JSON data, return as a dataframe.'''
    cache_path = '{}{}.pkl'.format(data_dir, file_name)
    try:        
        f = open(cache_path, 'rb')
        df = pickle.load(f)   
        print('Loaded {} from cache'.format(file_name))
    except (OSError, IOError) as e:
        print('Downloading {}'.format(json_url))
        df = pd.read_json(json_url)
        df.to_pickle(cache_path)
        print('Cached response at {}'.format(cache_path))
    return df

Next, we'll define a function to format Poloniex API HTTP requests and call our new get_json_data function to save the resulting data.

In [32]:
base_polo_url = 'https://poloniex.com/public?command=returnChartData&currencyPair={}&start={}&end={}&period={}'
start_date = datetime.strptime('2015-01-01', '%Y-%m-%d') # get data from the start of 2015

end_date = datetime.now() # up until today
period = 86400 # pull daily data (86,400 seconds per day)

def get_crypto_data(poloniex_pair):
    '''Retrieve cryptocurrency data from poloniex'''
    json_url = base_polo_url.format(poloniex_pair, start_date.timestamp(), end_date.timestamp(), period)
    data_df = get_json_data(json_url, poloniex_pair)
    data_df = data_df.set_index('date')
    return data_df

This function will take a cryptocurrency pair string (such as 'BTC_ETH') and return the dataframe containing the historical exchange rate of the two currencies.

## Download Trading Data From Poloniex
Most altcoins cannot be bought directly with USD; to acquire these coins individuals often buy Bitcoins and then trade the Bitcoins for altcoins on cryptocurrency exchanges.  For this reason we'll be downloading the exchange rate to BTC for each coin, and then we'll use our existing BTC pricing data to convert this value to USD.

We'll download exchange data for nine of the top cryptocurrencies -
[Ethereum](https://www.ethereum.org/), [Litecoin](https://litecoin.org/), [Ripple](https://ripple.com/), [Ethereum Classic](https://ethereumclassic.github.io/), [Stellar](https://www.stellar.org/), [Dashcoin](http://dashcoin.info/), [Siacoin](http://sia.tech/), [Monero](https://getmonero.org/), and [NEM](https://www.nem.io/).

In [112]:
altcoins = ['XRP', 'ETH', 'STR', 'NXT', 'XEM', 'XMR',
        'BCH', 'LTC', 'ETC', 'SC', 'LSK', 'MAID',
        'BTS', 'SYS', 'DOGE', 'STRAT', 'DASH', 'ZRX',
        'DGB', 'STEEM', 'OMG', 'FCT', 'REP', 'ZEC',
        'XCP', 'BCN', 'ARDR', 'GNT', 'BLK', 'GAS',
        'GAME', 'CVC', 'SBD', 'EMC2', 'BURST', 'DCR',
        'VRC', 'PASC', 'VTC', 'STORJ', 'LBC', 'NAV',
        'POT', 'OMNI', 'GNO', 'AMP', 'XPM', 'EXP',
        'VIA', 'NMC', 'BELA', 'NEOS', 'RIC', 'FLO',
        'BCY', 'BTCD', 'FLDC', 'RADS', 'PPC', 'NXC',
        'GRC', 'PINK', 'CLAM', 'XVC', 'XBC', 'BTM', 'HUC']

altcoin_data = {}
for altcoin in altcoins:
    coinpair = 'BTC_{}'.format(altcoin)
    crypto_price_df = get_crypto_data(coinpair)
    altcoin_data[altcoin] = crypto_price_df

Loaded BTC_XRP from cache
Loaded BTC_ETH from cache
Loaded BTC_STR from cache
Loaded BTC_NXT from cache
Loaded BTC_XEM from cache
Loaded BTC_XMR from cache
Loaded BTC_BCH from cache
Loaded BTC_LTC from cache
Loaded BTC_ETC from cache
Loaded BTC_SC from cache
Loaded BTC_LSK from cache
Loaded BTC_MAID from cache
Loaded BTC_BTS from cache
Loaded BTC_SYS from cache
Loaded BTC_DOGE from cache
Loaded BTC_STRAT from cache
Loaded BTC_DASH from cache
Loaded BTC_ZRX from cache
Loaded BTC_DGB from cache
Loaded BTC_STEEM from cache
Loaded BTC_OMG from cache
Loaded BTC_FCT from cache
Loaded BTC_REP from cache
Loaded BTC_ZEC from cache
Loaded BTC_XCP from cache
Loaded BTC_BCN from cache
Loaded BTC_ARDR from cache
Loaded BTC_GNT from cache
Loaded BTC_BLK from cache
Loaded BTC_GAS from cache
Loaded BTC_GAME from cache
Loaded BTC_CVC from cache
Loaded BTC_SBD from cache
Loaded BTC_EMC2 from cache
Loaded BTC_BURST from cache
Loaded BTC_DCR from cache
Loaded BTC_VRC from cache
Loaded BTC_PASC from cache


Now we have a dictionary of 9 dataframes, each containing the historical daily average exchange prices between the altcoin and Bitcoin.

We can preview the last few rows of the Ethereum price table to make sure it looks ok.

In [113]:
altcoin_data['XEM']

,close,high,low,open,quoteVolume,volume,weightedAverage
date,,,,,,,
2015-03-31,1.600000e-06,1.020000e-04,7.100000e-07,2.500000e-06,5.961381e+07,97.319038,1.630000e-06
2015-04-01,9.800000e-07,1.870000e-06,7.000000e-07,1.580000e-06,2.384865e+08,264.991041,1.110000e-06
2015-04-02,1.230000e-06,1.300000e-06,9.100000e-07,1.000000e-06,9.577869e+07,112.841565,1.170000e-06
2015-04-03,1.220000e-06,1.310000e-06,1.150000e-06,1.220000e-06,6.581851e+07,81.295697,1.230000e-06
2015-04-04,1.070000e-06,1.250000e-06,9.900000e-07,1.240000e-06,8.535530e+07,92.888274,1.080000e-06
2015-04-05,8.900000e-07,1.120000e-06,8.400000e-07,1.090000e-06,1.088683e+08,102.780788,9.400000e-07
2015-04-06,1.130000e-06,1.160000e-06,8.800000e-07,8.900000e-07,8.084315e+07,88.110318,1.080000e-06
2015-04-07,9.600000e-07,1.120000e-06,9.200000e-07,1.120000e-06,5.678294e+07,57.056392,1.000000e-06
2015-04-08,9.300000e-07,1.020000e-06,9.100000e-07,9.700000e-07,2.971265e+07,28.261242,9.500000e-07


## Merge All Of The Pricing Data Into A Single Dataframe
Next, we will define a simple function to merge a common column of each dataframe into a new combined dataframe.

In [114]:
def merge_dfs_on_column(dataframes, labels, col):
    '''Merge a single column of each dataframe into a new combined dataframe'''
    series_dict = {}
    for index in range(len(dataframes)):
        series_dict[labels[index]] = dataframes[index][col]
        
    return pd.DataFrame(series_dict)

## Visualize The Pricing Datasets
The next logical step is to visualize how these pricing datasets compare.  For this, we'll define a helper function to provide a single-line command to compare each column in the dataframe on a graph using Plotly.

In [115]:
def df_scatter(df, title, seperate_y_axis=False, y_axis_label='', scale='linear', initial_hide=False):
    '''Generate a scatter plot of the entire dataframe'''
    label_arr = list(df)
    series_arr = list(map(lambda col: df[col], label_arr))
    
    layout = go.Layout(
        title=title,
        legend=dict(orientation="h"),
        xaxis=dict(type='date'),
        yaxis=dict(
            title=y_axis_label,
            showticklabels= not seperate_y_axis,
            type=scale
        )
    )
    
    y_axis_config = dict(
        overlaying='y',
        showticklabels=False,
        type=scale )
    
    visibility = 'visible'
    if initial_hide:
        visibility = 'legendonly'
        
    # Form Trace For Each Series
    trace_arr = []
    for index, series in enumerate(series_arr):
        trace = go.Scatter(
            x=series.index, 
            y=series, 
            name=label_arr[index],
            visible=visibility
        )
        
        # Add seperate axis for the series
        if seperate_y_axis:
            trace['yaxis'] = 'y{}'.format(index + 1)
            layout['yaxis{}'.format(index + 1)] = y_axis_config    
        trace_arr.append(trace)

    fig = go.Figure(data=trace_arr, layout=layout)
    py.iplot(fig)

# Chart the prices

In [116]:
# Merge price of each altcoin into single dataframe 
combined_df = merge_dfs_on_column(list(altcoin_data.values()), list(altcoin_data.keys()), 'weightedAverage')

# Chart all of the altocoin prices
df_scatter(combined_df, 'Cryptocurrency Prices (BTC)', seperate_y_axis=False, y_axis_label='Coin Value (BTC)', scale='linear')

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


# Getting the data with featurese
adds features for classification. First computes the change ratio of all features. This way the features become unitless. Then find the target labels by shifting the weightedAverage and comparing with average value.

In [123]:
def getData(altcoin):
    if altcoin == 'Visa':
        df = visa_stock
    elif altcoin == 'BTC':
        df = btc_usd_df
    else:
        # selecting the features of the given altcoin
        df = altcoin_data[altcoin][['open', 'close', 'high', 'low', 'quoteVolume', 'volume', 'weightedAverage']]

    # renaming the weightedAverage column
    df = df.rename(columns={'weightedAverage': 'price'})

    df['btc'] = btc_usd_df['weightedAverage']
    

    # disable panda warning
    pd.options.mode.chained_assignment = None  # default='warn'


    df['price+'] = df['price'].shift(-1)
    
    

    # remove the last sample since it does not have a tomorow sample
    df = df.drop(df.index[-1])


    df['priceRate+'] = df['price+']/df['price'] - 1

    
    df['vol'] = df['volume']/df['quoteVolume']


    # adding previous days elements as new features
    n = 3
    for i in range(n):
        df['btc-%d'%(i+1)] = df['btc'].shift(i+1)
        df['price-%d'%(i+1)] = df['price'].shift(i+1)
        df['close-%d'%(i+1)] = df['close'].shift(i+1)
        df['open-%d'%(i+1)] = df['open'].shift(i+1)
        df['vol-%d'%(i+1)] = df['vol'].shift(i+1)
        df['totVol-%d'%(i+1)] = df['quoteVolume'].shift(i+1)


    # remove the first n samples since they don't have previous day data
    df = df.drop(df.index[0:n])


    df['btcRate-2'] = df['btc-2']/df['btc-3'] - 1
    df['btcRate-1'] = df['btc-1']/df['btc-2'] - 1
    df['btcRate'] = df['btc']/df['btc-1'] - 1
    df['btcAcc'] = df['btcRate'] - df['btcRate-1']
    df['btcAcc-1'] = df['btcRate-1'] - df['btcRate-2']
    df['btcJerk'] = df['btcAcc'] - df['btcAcc-1']


    df['volRate-2'] = df['vol-2']/df['vol-3'] - 1
    df['volRate-1'] = df['vol-1']/df['vol-2'] - 1
    df['volRate'] = df['vol']/df['vol-1'] - 1
    df['volAcc'] = df['volRate'] - df['volRate-1']
    df['volAcc-1'] = df['volRate-1'] - df['volRate-2']
    df['volJerk'] = df['volAcc'] - df['volAcc-1']


    df['totVolRate-2'] = df['totVol-2']/df['totVol-3'] - 1
    df['totVolRate-1'] = df['totVol-1']/df['totVol-2'] - 1
    df['totVolRate'] = df['quoteVolume']/df['totVol-1'] - 1
    df['totVolAcc'] = df['totVolRate'] - df['totVolRate-1']
    df['totVolAcc-1'] = df['totVolRate-1'] - df['totVolRate-2']
    df['totVolJerk'] = df['totVolAcc'] - df['totVolAcc-1']



    df['closeRate-2'] = df['close-2']/df['close-3'] - 1
    df['closeRate-1'] = df['close-1']/df['close-2'] - 1
    df['closeRate'] = df['close']/df['close-1'] - 1
    df['closeAcc'] = df['closeRate'] - df['closeRate-1']
    df['closeAcc-1'] = df['closeRate-1'] - df['closeRate-2']
    df['closeJerk'] = df['closeAcc'] - df['closeAcc-1']


    df['openRate-2'] = df['open-2']/df['open-3'] - 1
    df['openRate-1'] = df['open-1']/df['open-2'] - 1
    df['openRate'] = df['open']/df['open-1'] - 1
    df['openAcc'] = df['openRate'] - df['openRate-1']
    df['openAcc-1'] = df['openRate-1'] - df['openRate-2']
    df['openJerk'] = df['openAcc'] - df['openAcc-1']


    df['priceRate-2'] = df['price-2']/df['price-3'] - 1
    df['priceRate-1'] = df['price-1']/df['price-2'] - 1
    df['priceRate'] = df['price']/df['price-1'] - 1
    df['priceAcc'] = df['priceRate'] - df['priceRate-1']
    df['priceAcc-1'] = df['priceRate-1'] - df['priceRate-2']
    df['priceJerk'] = df['priceAcc'] - df['priceAcc-1']

    df['priceRange'] = df['high']/df['low'] - 1

    df['priceSwing'] = df['close']/df['open'] - 1
    
    
    
    # For adding technical indicators
    raw = df[['open', 'high', 'low', 'close', 'volume']]

    df.head()
    functions = talib.get_functions()
    remove_list = ['MAVP']
    for item in remove_list:
        functions.remove(item)
    

    data = df[['priceRate+',
                'low', 'high', 'priceRange',
               'open', 'close', 'priceSwing',
               'btc', 'btcRate', 'btcAcc', 'btcJerk',
              'vol', 'volRate', 'volAcc', 'volJerk',
              'price', 'priceRate', 'priceAcc', 'priceJerk',
              'quoteVolume', 'totVolRate', 'totVolAcc', 'totVolJerk']]
    for func in functions:
        outputs = eval(func)(raw)

        if (type(outputs) is pd.Series):
            data[func] = outputs
    
    n = 150
    # remove the first n samples
#     data = data.drop(data.index[0:n])
    data = data.dropna(axis=0, how='any')

    
    
    
    X = (data.drop(['priceRate+'], axis = 1)).values
    X = np.nan_to_num(X)
#     print('Shape of X: ',X.shape)
    tomorrrowPriceChange = data['priceRate+'].values
    tomorrrowPriceChange = np.nan_to_num(tomorrrowPriceChange)
    
    # finds the median weighted price changes
    priceChangeMed = np.median(tomorrrowPriceChange)
#     print('Median daily price change: %0.4f'%priceChangeMed)
    # computes the target label as whether the tomorrow value exceeds the median
    # in order to make the dataset balanced, i.e., number of pos and neg labels be equal
    y = (tomorrrowPriceChange > priceChangeMed)*1


#     delta = 0.00
#     y = (np.sign(tomorrrowPriceChange - delta) + np.sign(tomorrrowPriceChange + delta)) / 2.0

#     buyCount = np.sum(y==1)
#     sellCount = np.sum(y==-1)
#     holdCount = np.sum(y==0)

    # print(buyCount)
    # print(sellCount)
    # print(holdCount)
    
    
#     y = (tomorrrowPriceChange > delta)

    return X, y, tomorrrowPriceChange, X.shape[0], priceChangeMed

# Preprocessing

In [124]:
def preprocess(X, y, testSize, k):
        
    X_train = X[0:-(testSize-1), :]
    y_train = y[0:-(testSize-1)]
    
    # normalize the dataFrame
    min_max_scaler = preprocessing.MinMaxScaler()
    X_train = min_max_scaler.fit_transform(X_train)
    X = min_max_scaler.fit_transform(X)

    # feature selection
    selector = SelectKBest(mutual_info_classif, k=100)
    X_train = selector.fit_transform(X_train, y_train)
    X = selector.transform(X)
    
    # dimensionality reduction
    pca = decomposition.PCA(n_components=k)
    pca.fit(X)
    X = pca.transform(X)
    
    return X

# Correlation between the best indicators

In [125]:
# correlation_matrix = data.corr()
# plt.figure(figsize=(20,16))
# ax = sns.heatmap(correlation_matrix, vmax=1, square=True,annot=True,cmap='RdYlGn')
# plt.title('Correlation matrix between the features')
# plt.show()

# Model selection

In [126]:

dict_classifiers = {
    "Logistic Regression": LogisticRegression(),
#     "Nearest Neighbors": KNeighborsClassifier(),
    "Linear SVM": SVC(),
#     "Gradient Boosting Classifier": GradientBoostingClassifier(),
#     "Decision Tree": tree.DecisionTreeClassifier(),
#     "Random Forest": RandomForestClassifier(n_estimators = 18),
#     "Neural Net": MLPClassifier(alpha = 1),
#     "Naive Bayes": GaussianNB(),
#     "Bagging-kNN": BaggingClassifier(KNeighborsClassifier(), max_samples=0.5, max_features=0.8),
#     "AdaBoost": AdaBoostClassifier(n_estimators=100),
#     "Auto-sklearn": autosklearn.classification.AutoSklearnClassifier(),
#     "Voting": VotingClassifier(estimators=[
#                                            ('1',  LogisticRegression()), 
#                                            ('0', SVC())], voting='hard')
}


no_classifiers = len(dict_classifiers.keys())
def batch_classify(X, y, testSize, verbose = True):
    df_results = pd.DataFrame(data=np.zeros(shape=(no_classifiers,5)),
                              columns = ['classifier', 'train_score', 'test_score', 'baseline', 'time'])
    count = 0
    for key, classifier in dict_classifiers.items():
        t_start = time.clock()
        
        train_scores = np.zeros(testSize)
        test_scores = np.zeros(testSize)
        y_preds = np.zeros(testSize)
        for i in range(testSize):
            X_train = X[0:-(i+1), :]
            y_train = y[0:-(i+1)]
            
            X_test = X[-(i+1), :].reshape(1, -1)
            y_test = y[-(i+1)].reshape(1, -1)
            
            classifier.fit(X_train, y_train)
            train_scores[i] = classifier.score(X_train, y_train)
            test_scores[i] = classifier.score(X_test, y_test)
            y_preds[i] = classifier.predict(X_test)

#         average_precision = average_precision_score(Y_test, y_pred)
#         precision = precision_score(Y_test, y_pred)

        t_end = time.clock()
        t_diff = t_end - t_start
        
        df_results.loc[count,'classifier'] = key
        df_results.loc[count,'train_score'] = np.mean(train_scores)
        df_results.loc[count,'test_score'] = np.mean(test_scores)
        df_results.loc[count,'time'] = t_diff
        
#         df_results.loc[count,'precision'] = precision
#         df_results.loc[count,'avg_precision'] = average_precision

        y_test = y[testSize:]
        baseline = np.sum(y_test==np.argmax([np.sum(y_test==0),
                                             np.sum(y_test==1)]))/np.size(y_test)
        df_results.loc[count,'baseline'] = baseline
        
        count+=1
    return df_results


In [127]:
coin = 'LTC'
testSize = 200
X, y, y_cont, size, median = getData(coin)

X = preprocess(X, y, testSize, k=20)

df_results = batch_classify(X, y, testSize)
display(df_results.sort_values(by='test_score', ascending=False))

,classifier,train_score,test_score,baseline,time
0,Logistic Regression,0.652268,0.68,0.510328,1.959570
1,Linear SVM,0.644547,0.68,0.510328,19.970676


# Altcoin selection

In [129]:
coins = ['XRP', 'ETH', 'STR', 'NXT', 'XEM', 'XMR']
#         'LTC', 'ETC', 'SC', 'LSK', 'MAID',
#         'BTS', 'SYS', 'DOGE', 'STRAT', 'DASH',
#         'DGB', 'STEEM', 'FCT', 'REP', 'ZEC',
#         'XCP', 'BCN', 'ARDR', 'GNT', 'BLK',
#         'GAME', 'SBD', 'EMC2', 'BURST', 'DCR',
#         'VRC', 'PASC', 'VTC', 'LBC', 'NAV',
#         'POT', 'OMNI', 'AMP', 'XPM', 'EXP',
#         'VIA', 'NMC', 'BELA', 'NEOS', 'RIC', 'FLO',
#         'BCY', 'BTCD', 'FLDC', 'RADS', 'PPC', 'NXC',
#         'GRC', 'PINK', 'CLAM', 'XVC', 'XBC', 'BTM', 'HUC']
testSize = 2
df_results = pd.DataFrame(data=np.zeros(shape=(len(coins), 6)),
                              columns = ['altcoin', 'days', 'median', 'train_score', 'test_score', 'baseline'])

classifier = LogisticRegression()
count = 0
for coin in coins:
#     print(coin)

    X, y, y_cont, size, median = getData(coin)
    X = preprocess(X, y, testSize, k=20)
    
    t_start = time.clock()

    train_scores = np.zeros(testSize)
    test_scores = np.zeros(testSize)
    y_preds = np.zeros(testSize)
    for i in range(testSize):
        X_train = X[0:-(i+1), :]
        y_train = y[0:-(i+1)]

        X_test = X[-(i+1), :].reshape(1, -1)
        y_test = y[-(i+1)].reshape(1, -1)

        classifier.fit(X_train, y_train)
        train_scores[i] = classifier.score(X_train, y_train)
        test_scores[i] = classifier.score(X_test, y_test)
        y_preds[i] = classifier.predict(X_test)


    t_end = time.clock()
    t_diff = t_end - t_start

    df_results.loc[count,'altcoin'] = coin
    df_results.loc[count,'days'] = size
    df_results.loc[count,'median'] = median
    df_results.loc[count,'train_score'] = np.mean(train_scores)
    df_results.loc[count,'test_score'] = np.mean(test_scores)
    df_results.loc[count,'time'] = t_diff


    y_test = y[testSize:]
    baseline = np.sum(y_test==np.argmax([np.sum(y_test==0),
                                         np.sum(y_test==1)]))/np.size(y_test)
    df_results.loc[count,'baseline'] = baseline

    count+=1
    
    
display(df_results.sort_values(by='test_score', ascending=False))

,altcoin,days,median,train_score,test_score,baseline,time
0,XRP,1023.0,-0.004213,0.653940,1.0,0.500490,0.033917
2,STR,1023.0,-0.005764,0.639256,1.0,0.500490,0.023532
3,NXT,1023.0,-0.006381,0.637298,1.0,0.500490,0.017480
4,XEM,934.0,-0.004029,0.630563,1.0,0.500000,0.016451
1,ETH,804.0,-0.002174,0.684112,0.0,0.501247,0.021618
5,XMR,1023.0,-0.001491,0.666667,0.0,0.500490,0.017553
